In [1]:
import csv
import json
import datetime
import pandas as pd
from gensim.models import phrases 
from gensim import corpora, models, similarities #calc all similarities at once, from http://radimrehurek.com/gensim/tut3.html
from scipy import spatial
from gensim.models import Word2Vec, KeyedVectors
import requests

In [94]:
with open("triplets_training_data.csv", 'r') as csvfile:
    csvdata = csv.reader(csvfile)
    with open('triplets_training_data2.csv', 'w', newline ="") as csvfile2:
        writer = csv.writer(csvfile2)
        for row in csvdata:
            if row[1] == 'Brooklyn':
                row[1] = 'Kings'
            row.append(mydict[row[1]])
            writer.writerow(row)

In [7]:
with open("county-to-states.csv", 'r') as csvfile:
    reader = csv.reader(csvfile)
    mydict = {rows[0]:rows[1] for rows in reader}

with open("county-fips-to-name.csv", 'r') as csvfile2:
    reader = csv.reader(csvfile2)
    mydict2 = {rows[1]:rows[0] for rows in reader}

In [79]:
with open('C:/Users/marce/Downloads/Coronavirus_social_distancing_data/06/03/2020-06-03-social-distancing.json') as jfile:
    data = json.load(jfile)
    print(data)

{'1001': {'median_non_home_dwell_time': 200.25}, '1003': {'median_non_home_dwell_time': 169.9784946237}, '1005': {'median_non_home_dwell_time': 171.6086956522}, '1007': {'median_non_home_dwell_time': 220.9333333333}, '1009': {'median_non_home_dwell_time': 224.8}, '1011': {'median_non_home_dwell_time': 135.4285714286}, '1013': {'median_non_home_dwell_time': 174.3157894737}, '1015': {'median_non_home_dwell_time': 189.3595505618}, '1017': {'median_non_home_dwell_time': 165.303030303}, '1019': {'median_non_home_dwell_time': 220.6315789474}, '1021': {'median_non_home_dwell_time': 217.6333333333}, '1023': {'median_non_home_dwell_time': 188.8666666667}, '1025': {'median_non_home_dwell_time': 186.0}, '1027': {'median_non_home_dwell_time': 209.5384615385}, '1029': {'median_non_home_dwell_time': 249.2727272727}, '1031': {'median_non_home_dwell_time': 197.8484848485}, '1033': {'median_non_home_dwell_time': 211.2173913043}, '1035': {'median_non_home_dwell_time': 153.8333333333}, '1037': {'median_n

In [261]:
test = 'https://api.census.gov/data/2017/cbp'

url = 'https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_PT,NUI_PT&for=county:195&in=state:02&time=2017&key=22675759fc334d9c218ccd6b585baaaa0bc40194'
url2 = 'https://api.census.gov/data/timeseries/poverty/saipe?get=SAEPOVALL_PT,SAEMHI_PT&for=county:001&in=state:04&time=2018&key=22675759fc334d9c218ccd6b585baaaa0bc40194'
url3 = 'https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_PT,NUI_PT&for=county:001&in=state:04&time=2017&key=22675759fc334d9c218ccd6b585baaaa0bc40194'
response = requests.get(url).json()
response2 = requests.get(url2).json()
response3 = requests.get(url3).json()

print(response3[1][0])
pov1 = response[1][0]
pov2 = response[1][1]
pov3 = response2[1][0]
pov4 = response2[1][1]
print(pov1, pov2, pov3, pov4)

[['NIC_PT', 'NUI_PT', 'time', 'state', 'county'], ['48245', '11444', '2017', '04', '001']]
2165 499 26435 33652


In [123]:
zipdict = dict()
with open('C:/Users/marce/Downloads/ZIP_COUNTY_032020.csv', 'r') as csvfile:
    csvdata = csv.reader(csvfile)
    for row in csvdata:
        if row[1] not in zipdict:
            zipdict[row[1]] = row[0]
            
            
dict4 = dict()
with open('C:/Users/marce/Documents/Insight_Project/Project/data/raw/list_of_counties.csv', 'r') as file2:
            for j in file2:
                city = j[:-1]
                if city == 'Brooklyn':
                    city = 'Kings'
                code = mydict2[city]
                if len(code) == 4:
                    code2 = '0' + code
                else:
                    code2 = code
                zipcode = zipdict[code2]
                for date in ['2020-06-03', '2020-05-27', '2020-05-20']:
                    url = 'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?q={}&date={}&tp=24&format=json&key=0b174fb5449740019bc191210201206'.format(zipcode, date)
                    response = requests.get(url).json()
                    temp = response['data']['weather'][0]['maxtempC']
                    temp2 = response['data']['weather'][0]['avgtempC']
                    sun = response['data']['weather'][0]['sunHour']
                    wind = response['data']['weather'][0]['hourly'][0]['windspeedKmph']
                    rain = response['data']['weather'][0]['hourly'][0]['precipMM']
                    dict4[code+','+date]=(temp, temp2, sun, wind, rain)

In [264]:
def county_data2():
    mydict3 = dict()
    with open('C:/Users/marce/Documents/Insight_Project/Project/data/raw/list_of_counties.csv', 'r') as file2:
            for j in file2:
                city = j[:-1]
                if city == 'Brooklyn':
                    city = 'Kings'
                code = mydict2[city]
                if len(code) == 4:
                    code2 = '0' + code
                else:
                    code2 = code
                    
                url = 'https://api.census.gov/data/2017/pep/population?get=DENSITY,POP&for=county:{}&in=state:{}&key=22675759fc334d9c218ccd6b585baaaa0bc40194'.format(code2[2:], code2[:2])   
                url2 = 'https://api.census.gov/data/timeseries/healthins/sahie?get=NIC_PT,NUI_PT,NAME&for=county:{}&in=state:{}&time=2017&key=22675759fc334d9c218ccd6b585baaaa0bc40194'.format(code2[2:], code2[:2])
                url3 = 'https://api.census.gov/data/timeseries/poverty/saipe?get=SAEPOVALL_PT,SAEMHI_PT,NAME&for=county:{}&in=state:{}&time=2018&key=22675759fc334d9c218ccd6b585baaaa0bc40194'.format(code2[2:], code2[:2])
                
                response = requests.get(url).json()
                response2 = requests.get(url2).json()
                response3 = requests.get(url3).json()

                density = response[1][0]
                population = response[1][1]
                healthins1 = response2[1][0]
                healthins2 = response2[1][1]
                pov1 = response3[1][0]
                pov2 = response3[1][1]
                
                mydict3[code] = (density, population, healthins1, healthins2, pov1, pov2)
            return mydict3
    

mydict3 = county_data2()
print(mydict3)

{'6037': ('2504.4612193', '10163507', '7798975', '872284', '1409155', '67986'), '27031': ('3.716683247', '5398', '3555', '324', '511', '54465'), '48201': ('2729.1419', '4652980', '3210664', '908742', '767367', '60241'), '4013': ('468.19534044', '4307033', '3175198', '436313', '535183', '65234'), '6073': ('793.16750363', '3337685', '2565369', '235409', '372148', '78777'), '51137': ('105.75064556', '36073', '24997', '3305', '3387', '63681'), '12086': ('1449.3333861', '2751796', '1811760', '453444', '436103', '52043'), '48113': ('3002.1071445', '2618148', '1772552', '530742', '368310', '59838'), '36047': ('38182.229746', '2648771', '2078148', '179878', '483632', '60862'), '6065': ('336.26486829', '2423266', '1866185', '183037', '307511', '66793'), '36081': ('21685.341676', '2358582', '1784061', '202448', '261819', '68432'), '53033': ('1034.7198771', '2188649', '1776493', '118292', '202628', '94822'), '55019': ('28.667578984', '34679', '23736', '4677', '4052', '55245'), '6071': ('107.56212

In [55]:
def new_cases_week(county, date, data):
    date2 = str(datetime.datetime.strptime(date, "%Y-%m-%d") - datetime.timedelta(days = 7))[:10]
    while True:
        try:
            cases = data[county]['dates'][date]['cases'] - data[county]['dates'][date2]['cases']
            break
        except KeyError:
            date = str(datetime.datetime.strptime(date, "%Y-%m-%d") - datetime.timedelta(days = 1))[:10]
            date2 = str(datetime.datetime.strptime(date2, "%Y-%m-%d") - datetime.timedelta(days = 1))[:10]
            continue
    return cases

In [152]:
def county_data(code, date):
    demographics = mydict3[code]
    demographics = [float(d) for d in demographics]

    weather = dict4[code+','+date]
    weather = [float(w) for w in weather]
    return weather, demographics

In [153]:
def social_dist_data(code, date):
    directory = 'C:/Users/marce/Downloads/Coronavirus_social_distancing_data/' + date[5:7] + '/' + date[8:] + '/' + date + '-social-distancing.json'
    with open(directory, 'r') as jfile:
        data = json.load(jfile)
        social_dist = data[code]['median_non_home_dwell_time']
    return social_dist
    # df.loc[df['origin_census_block_group'][:5] == code, 'median_non_home_dwell_time'].mean()

In [265]:
currentmodel = Word2Vec.load('newsdata_model2')
features = list()
error = list()
scores = list()
activities = list()
dates = list()
counties = list()

def feature_extraction(currentmodel):
    with open('triplets_training_data2.csv', 'r') as csvfile:
        csvdata = csv.reader(csvfile)
        directory = 'C:/Users/marce/Downloads/coronadatascraper-timeseries_by_location__202006110200__202006110200.json'
        with open(directory, 'r', encoding="utf8") as jf:
            data = json.load(jf)
            for i, row in enumerate(csvdata):
                date = row[2][:10]
                county = row[1]
                code = mydict2[county]
                county += ' County, ' + row[-1] + ', United States'
                activity = row[0].lower().replace(" ",'_')
                
                weather, demographics = county_data(code, date)
                distancing = social_dist_data(code, date)
                difference = new_cases_week(county, date, data)

                try:

                    sentiment = currentmodel.wv[activity] - currentmodel.wv['safe']
                    feature = np.append(sentiment, (float(distancing), int(difference)))
                    feature = np.concatenate((feature, np.array(weather).reshape(-1)))
                    feature = np.concatenate((feature, np.array(demographics).reshape(-1)))
                    feature = feature.reshape(-1)

                    if float(row[3]) >= 0:
                        scores.append(0)
                    else:
                        scores.append(1)
                    features.append(feature)
                    activities.append(activity)
                    counties.append(county)
                    dates.append(date)
                except KeyError:
                    pass
    return features, scores, activities, counties, dates
                    
features, scores, activities, counties, dates = feature_extraction(currentmodel)

In [266]:
import numpy as np
features = np.array(features)
print(features.shape)
data = pd.DataFrame(features)

# sentiment = ['sentiment' + str(i+1) for i in range(0,100)]
activities = np.array(activities)
data['activities'] = activities

counties = np.array(counties)
data['counties'] = counties

dates = np.array(dates)
data['dates'] = dates

scores = np.array(scores)
data['scores'] = scores

data.to_csv('triplet_training_data_with_features.csv')



corr_matrix = data.corr()
corr_matrix["scores"].sort_values(ascending=False)

(2749, 113)


scores    1.000000
92        0.132283
60        0.125602
62        0.114350
88        0.110281
            ...   
64       -0.095251
37       -0.097533
10       -0.101745
53       -0.103660
55       -0.104630
Name: scores, Length: 114, dtype: float64